![](https://image.slidesharecdn.com/karenfent-191031210734/95/enabling-biobankscale-genomic-processing-with-spark-sql-53-638.jpg?cb=1572556092)slideshare.net

#Variant Call Format (VCF)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# other imports
import sys

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#Code by Alistair Miles  http://alimanfoo.github.io/2018/04/09/selecting-variants.html

In [ ]:
!pip install scikit-allel

In [ ]:
import allel
allel.__version__

#Start with the scikit-allel function read_vcf()

In [ ]:
#callset = allel.read_vcf('../input/end-als/end-als/genomics-data/AnswerALS_subset_annovar.hg38_anno_and_geno.no_intergenic.vcf')

#That snippet above took so long (more than 15 minutes to read a VCF) and is consuming My RAM.
Then I commented it.

#The callset object returned by read_vcf() is a Python dictionary (dict). It contains several NumPy arrays, each of which can be accessed via a key. Here are the available keys:

In [ ]:
#sorted(callset.keys())

#The ‘samples’ array contains sample identifiers extracted from the header line in the VCF file.

In [ ]:
#callset['samples']

In [ ]:
import numpy as np
np.__version__

In [ ]:
!pip install zarr

In [ ]:
import zarr
zarr.__version__

In [ ]:
!pip install numcodecs

In [ ]:
import numcodecs
numcodecs.__version__

#Extract data from a VCF

In [ ]:
vcf_path = '../input/end-als/end-als/genomics-data/AnswerALS_subset_annovar.hg38_anno_and_geno.no_intergenic.vcf'
!ls -lh {vcf_path}

#Extract data and convert to Zarr format, use the `vcf_to_zarr`() function from scikit-allel.
That conversion will make life easier downstream.

In [ ]:
zarr_path = '../input/end-als/end-als/genomics-data/AnswerALS_subset_annovar.hg38_anno_and_geno.no_intergenic.vcf'

In [ ]:
#The original snippet used data from chromosome 22. End ALS didn't mention any chromossome in the file
#Trying to make it work, I removed the group and the fields. Unfortunately, it didn't work.

#allel.vcf_to_zarr(vcf_path, zarr_path, group='22', fields='*', log=sys.stdout, overwrite=True)

In [ ]:
allel.vcf_to_zarr(vcf_path, zarr_path, group='*', fields='*', log=sys.stdout, overwrite=True)

#After FSPathExistNotDir: path exists but is not a directory: %r  Everything went wrong.

#Maybe anyone can fix it.

#Open the Zarr data 

In [ ]:
callset = zarr.open_group(zarr_path, mode='r')
callset.tree(expand=True)

In [ ]:
gt_zarr = callset['22/calldata/GT']
gt_zarr.info

#Loading data for a gene

In [ ]:
pos = allel.SortedIndex(callset['22/variants/POS'])
pos

In [ ]:
loc_region = pos.locate_range(20000000, 20100000)
loc_region

In [ ]:
gt_region = allel.GenotypeArray(gt_zarr[loc_region])
gt_region

#Filtering variants

In [ ]:
multi_allelic = callset['22/variants/MULTI_ALLELIC'][:]
multi_allelic

#Extract genotype data for these variants

In [ ]:
gt = allel.GenotypeArray(gt_zarr)
gt

#Extract genotypes for the selection

In [ ]:
gt_variant_selection = gt.compress(loc_variant_selection, axis=0)
gt_variant_selection

#Use Dask if data is larger and/or your computer doesn’t have much RAM

In [ ]:
gt_dask = allel.GenotypeDaskArray(gt_zarr)
gt_dask

#Apply the selection, using almost the same syntax, except that when working via Dask we need to call the compute() method to get the final result:

In [ ]:
gt_variant_selection = gt_dask.compress(loc_variant_selection, axis=0).compute()
gt_variant_selection

#I hope to learn a little bit how to work with vcf files, scikit-allel and BioInformatics.